# M4 | Research Investigation Notebook

In this notebook, you will do a research investigation of your chosen dataset in teams. You will begin by formally selecting your research question (task 0), then processing your data (task 1), creating a predictive model (task 2), evaluating your model's results (task 3), and describing the contributions of each team member (task 4).

For grading, please make sure your notebook has all cells run and is stored in your team's [Github Classroom repository](https://classroom.github.com/a/CNxME27U). You will also need to write a short, 2 page report about your design decisions as a team, to be stored in your repository. The Milestone 4 submission will be the contents of your repository at the due date (April 28 at 23:59 CET).

## Brief overview of Lernnavi
[Lernnavi](https://www.lernnavi.ch) is an instrument for promoting part of the basic technical study skills in German and mathematics.

Lernnavi's dataset is formatted in three main tables:
* ***users***: demographic information of users.
* ***events***: events done by the users in the platform.
* ***transactions***: question and answer solved by user.

These table are described in detail in the [Milestone 2 data exploration notebook](https://github.com/epfl-ml4ed/mlbd-2023/blob/main/project/milestone-02/m2_lernnavi_sciper.ipynb). We have also provided access to a [test account to experiment with Lernnavi](https://moodle.epfl.ch/mod/forum/discuss.php?d=88094). 

You should provide arguments and justifications for all of your design decisions throughout this investigation. You can use your M3 responses as the basis for this discussion.

In [44]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = './data' # You many change the directory

users = pd.read_csv('{}/users.csv.gz'.format(DATA_DIR))
events = pd.read_csv('{}/events.csv.gz'.format(DATA_DIR))
transactions = pd.read_csv('{}/transactions.csv.gz'.format(DATA_DIR))
documents = pd.read_csv('{}/documents.csv.gz'.format(DATA_DIR))
topics_translated = pd.read_csv('{}/topics_translated.csv'.format(DATA_DIR)).rename(columns={"german_name": "challenge_name"})


## Task 0: Research Question

**Research question:**
Can we predict a user's mastery level in week N, by using the available data from week 1 to N-1?

## Task 1: Data Preprocessing

As we will be predicting the mastery level of each user per week, we first need to parse the available mastery level data.

We'll be using the NAVIGATE_DASHBOARD tasks to get the mastery level as we have found out that using the ACCEPT_PROGRESS and REJECT_PROGRESS gives us a lot less data to work with.

In [45]:
# Get the total mastery as the sum of the mastery of all topics
import json
from tqdm import notebook as vis
from numpy import mean as mean
from numpy import sum as sum
rows = []
total = events[events['action']=='NAVIGATE_DASHBOARD'].shape[0]
for index,row in vis.tqdm(events[events['action']=='NAVIGATE_DASHBOARD'].iterrows(), 
                          total=total, 
                          desc="Processing records"):
    json_loaded = json.loads(row['tracking_data'])
    if(json_loaded['trackingDataType'] != 'DASHBOARD_VIEW_DATA'):
        continue
    title = json_loaded['dashboard']['title']
    if(len(json_loaded['dashboard']['topics']) == 0):
        continue
    topics = json_loaded['dashboard']['topics']
    total_mastery = []
    total_diligence = []
    for topic in topics:
        children = topic['children']
        for child in children:
            total_mastery.append(topic['userData']['mastery'])
            total_diligence.append(topic['userData']['diligence'])
    user_id = row['user_id']
    start_time = row.event_date
    # add the row to the list
    rows.append([user_id, title, sum(total_mastery), start_time, sum(total_diligence)])

Processing records:   0%|          | 0/1093791 [00:00<?, ?it/s]

The below cell is commented out for the reason we have described above.

In [46]:
# from tqdm import notebook as vis
# rows2 = []
# total = events[(events['action']=='ACCEPT_PROGRESS') | (events['action'] == 'REJECT_PROGRESS')].shape[0]
# for index,row in vis.tqdm(events[(events['action']=='ACCEPT_PROGRESS') | (events['action'] == 'REJECT_PROGRESS')].iterrows(),
#                           total=total,
#                           desc="Processing records"):
#     json_loaded = json.loads(row['tracking_data'])
#     topics = json_loaded['sessionEndScreenTopics']
#     for topic in topics:
#         rows2.append([row['user_id'], topic['topic']['name'], topic['newMastery'], row['event_date']])
# print(rows2)

After parsing the mastery level data we have a list of the following form (user_id, title, mastery, timestamp)

Now we transform this into a pandas DataFrame

In [47]:
# Create new dataframe from the list
mastery_df = pd.DataFrame(rows, columns=['user_id', 'title', 'mastery', 'start_time' , 'diligence'])
# Find the earliest start time for each user in events table
earliest_start_time = events.groupby('user_id').event_date.min().reset_index()
# rename column to min_start_time
earliest_start_time = earliest_start_time.rename(columns={'event_date': 'min_start_time'})
# merge with earliest_transaction
mastery_df = mastery_df.merge(earliest_start_time, on='user_id', how='left')
# convert start_time to datetime
mastery_df.start_time = pd.to_datetime(mastery_df.start_time)
# convert earliest_transaction to datetime
mastery_df.min_start_time = pd.to_datetime(mastery_df.min_start_time)
# calculate the number of weeks since first transaction
mastery_df['weeks_since_first_transaction'] = (mastery_df.start_time - mastery_df.min_start_time).dt.days//7
# drop the columns start_time and min_start_time
mastery_df = mastery_df.drop(columns=['start_time','min_start_time'])
# A user can check their mastery multiple times a week,
# Find the max mastery for each user in each week, and find the max diligence for each user in each week
# Keep those rows and drop the rest
mastery_df = mastery_df.groupby(['user_id','title','weeks_since_first_transaction']).agg({'mastery': 'max', 'diligence': 'max'}).reset_index()
# Multiply mastery col by 10 as explained in the data metadata file
mastery_df.mastery = mastery_df.mastery*10

Mastery level for each user is different per topic, the 3 main topics that encompasses all other subtopics are Orthografie, Mathematik and Deutsch. We drop Orthografie for having almost no data related to it.

In [48]:
print(mastery_df.title.unique())
print('Orthografie: ' + str(mastery_df[mastery_df.title == 'Orthografie'].shape))
print('Mathematik: ' + str(mastery_df[mastery_df.title == 'Mathematik'].shape))
print('Deutsch: ' + str(mastery_df[mastery_df.title == 'Deutsch'].shape))
# Notice Orthografie has almost no rows.
# Drop rows that have title as 'Orthografie'
mastery_df = mastery_df[mastery_df.title != 'Orthografie']

['Deutsch' 'Mathematik' 'Orthografie']
Orthografie: (6, 5)
Mathematik: (34981, 5)
Deutsch: (44648, 5)


We split our data frame to two, one for each topic.

In [49]:
mastery_df_german = mastery_df[mastery_df['title'] == "Deutsch"]
mastery_df_math = mastery_df[mastery_df['title'] == "Mathematik"]

Now we have our initial table, we need to add some features to it so that it's suitable to be trained by an ML model.

The below cell adds the following features: 
    1. Weekly solved question count
    2. Weekly percentage of CORRECTLY solved questions, where partially correct questions count as 0.5 correct.

In [50]:
# Now we will add extra features to this table
"""
    CREATE THE TABLE
"""
new_transactions = transactions[['transaction_token','evaluation', 'document_id']]
new_events = events[['user_id', 'transaction_token', 'event_date', 'action']]
new_transactions = new_transactions.merge(new_events, on='transaction_token', how='right')
new_transactions = new_transactions.merge(earliest_start_time, on='user_id', how='left')
new_transactions.event_date = pd.to_datetime(new_transactions.event_date)
new_transactions.min_start_time = pd.to_datetime(new_transactions.min_start_time)
new_transactions['weeks_since_first_transaction'] = (new_transactions['event_date'] - new_transactions['min_start_time']).dt.days // 7

"""
    FIND WEEKLY EVENT COUNT
"""
# Removed event counts as it's hard to distinguish events between different topics.
# Find the number of transactions for each user in each week
#num_events_weekly = new_transactions.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
# Rename the column to num_events
#num_events_weekly = num_events_weekly.rename(columns={'action': 'num_events'})

"""
    FIND WEEKLY QUESTIONS SOLVED
"""
# Only consider question answering events (action = 'SUBMIT_ANSWER')
num_questions_weekly = new_transactions[new_transactions.action == 'SUBMIT_ANSWER']
num_questions_weekly = num_questions_weekly.dropna(subset = ["document_id"])
doc_to_topic = documents.merge(topics_translated, how='left', left_on='topic_id', right_on='id')[['document_id','math']]
doc_to_topic = doc_to_topic.drop_duplicates("document_id")
num_questions_weekly = num_questions_weekly.merge(doc_to_topic, how = 'left', on='document_id')

#Drop rows with math = NaN
num_questions_weekly = num_questions_weekly[num_questions_weekly['math'].notna()]

#Create german dataframe
num_questions_weekly_german = num_questions_weekly[num_questions_weekly['math'] == 0]
#Create math dataframe
num_questions_weekly_math = num_questions_weekly[num_questions_weekly['math'] == 1]
# Count the number of questions for each user in each week
num_questions_weekly_german = num_questions_weekly_german.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
num_questions_weekly_math = num_questions_weekly_math.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

# Rename column to num_questions
num_questions_weekly_math = num_questions_weekly_math.rename(columns={'action': 'num_questions'})
num_questions_weekly_german = num_questions_weekly_german.rename(columns={'action': 'num_questions'})

Now that we have the weekly number of questions solved for both german and mathematics, we tried to see if we have been able to capture all the questions solved(i.e. have we lost any data in the process of trying to split questions between topics?)

To do this:

We create the same dataframes again, but this time WITHOUT splitting it by topic, and calculate the number of TOTAL questions solved per week per user.

In [51]:
new_transactions_for_comparison = transactions[['transaction_token','evaluation', 'document_id']]
new_events_for_comparison = events[['user_id', 'transaction_token', 'event_date', 'action']]
new_transactions_for_comparison = new_transactions_for_comparison.merge(new_events_for_comparison, on='transaction_token', how='right')
new_transactions_for_comparison = new_transactions_for_comparison.merge(earliest_start_time, on='user_id', how='left')
new_transactions_for_comparison.event_date = pd.to_datetime(new_transactions_for_comparison.event_date)
new_transactions_for_comparison.min_start_time = pd.to_datetime(new_transactions_for_comparison.min_start_time)
new_transactions_for_comparison['weeks_since_first_transaction'] = (new_transactions_for_comparison['event_date'] - new_transactions_for_comparison['min_start_time']).dt.days // 7

num_questions_weekly_orig = new_transactions_for_comparison[new_transactions.action == 'SUBMIT_ANSWER']
num_questions_weekly_orig = num_questions_weekly_orig.merge(doc_to_topic, how = 'left', on='document_id')
nan_math_count = num_questions_weekly_orig.math.isna().sum()
num_questions_weekly_orig = num_questions_weekly_orig.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

Now below we get per user and per week the number of TOTAL questions solved(column name = action) and the amount of german questions solved and amount of math questions solved.

In [52]:

math_german_compared = num_questions_weekly_german.merge(num_questions_weekly_math, how = 'outer', on=['user_id','weeks_since_first_transaction'])
math_german_compared = math_german_compared.rename(columns={'num_questions_x': 'german_questions', 'num_questions_y': 'math_questions'})
math_german_compared = math_german_compared.fillna(value={'math_questions': 0, 'german_questions': 0})
question_comparison=num_questions_weekly_orig.merge(math_german_compared, how='left', on=['user_id','weeks_since_first_transaction'])
question_comparison=question_comparison.fillna(0)
question_comparison.head()

,user_id,weeks_since_first_transaction,action,german_questions,math_questions
0,387604,14,1,1.0,0.0
1,387604,17,4,4.0,0.0
2,387604,18,2,2.0,0.0
3,387604,19,1,1.0,0.0
4,387604,21,1,1.0,0.0


Below we find the rows where the amount of german questions + amount of math questions is not equal to the total amount of questions solved:

In [53]:
not_equal_rows = question_comparison[question_comparison.action != question_comparison.german_questions + question_comparison.math_questions]
not_equal_rows

,user_id,weeks_since_first_transaction,action,german_questions,math_questions
20,387604,50,33,32.0,0.0
29,387605,6,50,29.0,17.0
32,387605,14,58,29.0,22.0
33,387605,16,48,23.0,23.0
38,387605,32,20,14.0,0.0
...,...,...,...,...,...
65401,431353,1,12,0.0,11.0
65402,431354,1,21,0.0,20.0
65404,431355,2,21,0.0,20.0
65561,431656,0,22,0.0,21.0


In [54]:
question_comparison[question_comparison.action == question_comparison.german_questions + question_comparison.math_questions]

,user_id,weeks_since_first_transaction,action,german_questions,math_questions
0,387604,14,1,1.0,0.0
1,387604,17,4,4.0,0.0
2,387604,18,2,2.0,0.0
3,387604,19,1,1.0,0.0
4,387604,21,1,1.0,0.0
...,...,...,...,...,...
65730,431999,0,4,0.0,4.0
65731,432001,0,6,0.0,6.0
65732,432014,0,9,0.0,9.0
65733,432016,0,5,0.0,5.0


We notice there are 62001 rows where the equality does hold and 3734 rows where the equality does not hold and most of the rows only differ in only a couple of questions. Now we find the total amount of questions missing: Which is equal to 6282.

In [55]:
(question_comparison['action'] - question_comparison['german_questions'] - question_comparison['math_questions']).sum()

6282.0

Because in german_questions and math_questions we only consider the rows that have math column as 0 or 1, we don't consider the math columns that have the value NaN, and there are in total:

In [56]:
nan_math_count

6282

And this is where our difference comes in. As this is not something we can fix as this is due to some rows having missing data, we accept this data loss as it's within acceptible range.

In [57]:
"""
    FIND WEEKLY CORRECT QUESTIONS SOLVED
"""
# Only consider question answering events that are correct (evaluation = 'CORRECT')
num_correct_weekly = new_transactions[(new_transactions.evaluation == 'CORRECT') & (new_transactions.action == 'SUBMIT_ANSWER')]
#Merge with documents and topics to separate german and math
num_correct_weekly = num_correct_weekly.merge(doc_to_topic, how = 'left', on='document_id')
#Drop rows with math = NaN
num_correct_weekly = num_correct_weekly[num_correct_weekly['math'].notna()]

#Create german dataframe
num_correct_weekly_german = num_correct_weekly.query('math == 0')
#Create math dataframe
num_correct_weekly_math = num_correct_weekly.query('math == 1')

# Count the number of questions for each user in each week
num_correct_weekly_german = num_correct_weekly_german.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
num_correct_weekly_math = num_correct_weekly_math.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

# Rename column to num_questions
num_correct_weekly_math = num_correct_weekly_math.rename(columns={'action': 'num_correct'})
num_correct_weekly_german = num_correct_weekly_german.rename(columns={'action': 'num_correct'})

"""
    FIND WEEKLY PARTIALLY CORRECT QUESTIONS SOLVED
"""
# Only consider question answering events that are correct (evaluation = 'PARTIAL')
num_partial_weekly = new_transactions[(new_transactions.evaluation == 'PARTIAL') & (new_transactions.action == 'SUBMIT_ANSWER')]
num_partial_weekly = num_partial_weekly.merge(doc_to_topic, how = 'left', on='document_id')
#Drop rows with math = NaN
num_partial_weekly = num_partial_weekly[num_partial_weekly['math'].notna()]

#Create german dataframe
num_partial_weekly_german = num_partial_weekly.query('math == 0')
#Create math dataframe
num_partial_weekly_math = num_partial_weekly.query('math == 1')

# Count the number of questions for each user in each week
num_partial_weekly_german = num_partial_weekly_german.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
num_partial_weekly_math = num_partial_weekly_math.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

# Rename column to num_questions
num_partial_weekly_math = num_partial_weekly_math.rename(columns={'action': 'num_partial'})
num_partial_weekly_german = num_partial_weekly_german.rename(columns={'action': 'num_partial'})


# Merge the three tables together for german
num_questions_weekly_german = num_questions_weekly_german.merge(num_correct_weekly_german, on=['user_id','weeks_since_first_transaction'], how='left')
num_questions_weekly_german = num_questions_weekly_german.merge(num_partial_weekly_german, on=['user_id','weeks_since_first_transaction'], how='left')
num_questions_weekly_german.fillna(0, inplace=True)

# Merge the three tables together for math
num_questions_weekly_math = num_questions_weekly_math.merge(num_correct_weekly_math, on=['user_id','weeks_since_first_transaction'], how='left')
num_questions_weekly_math = num_questions_weekly_math.merge(num_partial_weekly_math, on=['user_id','weeks_since_first_transaction'], how='left')
num_questions_weekly_math.fillna(0, inplace=True)

# Create new column percentage_correct = (num_correct +0.5*num_partial)/ num_questions
num_questions_weekly_german['percentage_correct'] = 100 * (num_questions_weekly_german.num_correct + 0.5*num_questions_weekly_german.num_partial)/num_questions_weekly_german.num_questions
# Drop the columns num_correct and num_partial
num_questions_weekly_german = num_questions_weekly_german.drop(columns=['num_correct','num_partial'])

# Create new column percentage_correct = (num_correct +0.5*num_partial)/ num_questions
num_questions_weekly_math['percentage_correct'] = 100 * (num_questions_weekly_math.num_correct + 0.5*num_questions_weekly_math.num_partial)/num_questions_weekly_math.num_questions
# Drop the columns num_correct and num_partial
num_questions_weekly_math = num_questions_weekly_math.drop(columns=['num_correct','num_partial'])

The below cell adds the following features:
    1. Review task count
    2. View task count
    3. Window visible ratio

Do note that in the data the names for these actions are reversed, i.e. the VIEW_QUESTION corresponds to reviewing a task and REVIEW_TASK corresponds to viewing a question.

In [58]:
"""
    FIND THE REVIEW TASK COUNT
"""
# Only consider question answering events (action = 'SUBMIT_ANSWER')
num_review_weekly = new_transactions[new_transactions.action == 'VIEW_QUESTION']
num_review_weekly = num_review_weekly.dropna(subset = ["document_id"])
num_review_weekly = num_review_weekly.merge(doc_to_topic, how = 'left', on='document_id')
#Drop rows with math = NaN
num_review_weekly = num_review_weekly[num_review_weekly['math'].notna()]

#Create german dataframe
num_review_weekly_german = num_review_weekly.query('math == 0')
#Create math dataframe
num_review_weekly_math = num_review_weekly.query('math == 1')
# Count the number of questions for each user in each week
num_review_weekly_german = num_review_weekly_german.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
num_review_weekly_math = num_review_weekly_math.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

# Rename column to num_review
num_review_weekly_math = num_review_weekly_math.rename(columns={'action': 'num_review'})
num_review_weekly_german = num_review_weekly_german.rename(columns={'action': 'num_review'})


"""
    FIND THE VIEW COUNT
"""
# Only consider question answering events (action = 'SUBMIT_ANSWER')
num_view_weekly = new_transactions[new_transactions.action == 'REVIEW_TASK']
num_view_weekly = num_view_weekly.dropna(subset = ["document_id"])
num_view_weekly = num_view_weekly.merge(doc_to_topic, how = 'left', on='document_id')
#Drop rows with math = NaN
num_view_weekly = num_view_weekly[num_view_weekly['math'].notna()]

#Create german dataframe
num_view_weekly_german = num_view_weekly.query('math == 0')
#Create math dataframe
num_view_weekly_math = num_view_weekly.query('math == 1')

# Count the number of questions for each user in each week
num_view_weekly_german = num_view_weekly_german.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()
num_view_weekly_math = num_view_weekly_math.groupby(['user_id','weeks_since_first_transaction']).action.count().reset_index()

# Rename column to num_review
num_view_weekly_math = num_view_weekly_math.rename(columns={'action': 'num_view'})
num_view_weekly_german = num_view_weekly_german.rename(columns={'action': 'num_view'})

"""
FIND THE WINDOW VISIBLE RATIO FOR EACH USER
"""

# in the events table for each user find the count of action='WINDOW_VISIBLE_FALSE'
# and action='WINDOW_VISIBLE_TRUE'
# and then calculate the ratio of WINDOW_VISIBLE_TRUE / (WINDOW_VISIBLE_TRUE + WINDOW_VISIBLE_FALSE)

# drop all rows except those with action = 'WINDOW_VISIBLE_FALSE'
temp = new_transactions[new_transactions.action == 'WINDOW_VISIBLE_FALSE']

# count the number of rows for each user
num_window_visible_false = temp.groupby(['user_id','weeks_since_first_transaction']).action.count()
# fill the missing user ids with 0
num_window_visible_false = num_window_visible_false.to_frame().fillna(0).reset_index()
# drop all rows except those with action = 'WINDOW_VISIBLE_TRUE'
temp = new_transactions[new_transactions.action == 'WINDOW_VISIBLE_TRUE']

# count the number of rows for each user
num_window_visible_true = temp.groupby(['user_id','weeks_since_first_transaction']).action.count()

# fill the missing user ids with 0
num_window_visible_true = num_window_visible_true.to_frame().fillna(0).reset_index()

# Create a new dataframe with 2 columns: user_id and the ratio
df_window_visible = pd.DataFrame({'user_id': num_window_visible_true.user_id, 'weeks_since_first_transaction': num_window_visible_true.weeks_since_first_transaction, 'ratio_window_visible': num_window_visible_true.action / (num_window_visible_true.action + num_window_visible_false.action)})

"""
    MERGE ALL THE TABLES
"""
# Merge the tables together for german
#mastery_df = mastery_df.merge(num_events_weekly, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_german = mastery_df_german.merge(num_questions_weekly_german, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_german = mastery_df_german.merge(num_review_weekly_german, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_german = mastery_df_german.merge(num_view_weekly_german, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_german = mastery_df_german.merge(df_window_visible, on=['user_id','weeks_since_first_transaction'], how='left')
# Fill the NaN values in num_question and num_events with 0
mastery_df_german[['num_questions', 'num_review']] = mastery_df_german[['num_questions', 'num_review']].fillna(0)

# Merge the tables together for math
#mastery_df = mastery_df.merge(num_events_weekly, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_math = mastery_df_math.merge(num_questions_weekly_math, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_math = mastery_df_math.merge(num_review_weekly_math, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_math = mastery_df_math.merge(num_view_weekly_math, on=['user_id','weeks_since_first_transaction'], how='left')
mastery_df_math = mastery_df_math.merge(df_window_visible, on=['user_id','weeks_since_first_transaction'], how='left')
# Fill the NaN values in num_question and num_events with 0
mastery_df_math[['num_questions', 'num_review']] = mastery_df_math[['num_questions', 'num_review']].fillna(0)

#mastery_df_german["num_questions"] = mastery_df_german["num_questions"].astype(int)
#mastery_df["num_math_questions"] = mastery_df["num_math_questions"].as"ype(int)
#mastery_df["num_deutsch_questions"] = mastery_df["num_deutsch_questions"].astype(int)

In [59]:
mastery_df_math.head(10)

,user_id,title,weeks_since_first_transaction,mastery,diligence,num_questions,percentage_correct,num_review,num_view,ratio_window_visible
0,387604,Mathematik,5,0.000000,0.0,0.0,NaN,5.0,NaN,NaN
1,387604,Mathematik,54,1.041667,0.0,3.0,33.333333,0.0,12.0,NaN
2,387604,Mathematik,64,1.041667,0.0,0.0,NaN,0.0,NaN,0.625000
3,387604,Mathematik,75,1.041667,0.0,0.0,NaN,0.0,NaN,0.500000
4,387605,Mathematik,3,0.000000,3550.0,1.0,50.000000,2.0,NaN,NaN
5,387605,Mathematik,4,0.000000,3550.0,0.0,NaN,1.0,NaN,NaN
6,387605,Mathematik,6,0.000000,3970.0,17.0,76.470588,30.0,15.0,0.860465
7,387605,Mathematik,8,0.000000,3970.0,0.0,NaN,0.0,2.0,0.357143
8,387605,Mathematik,11,50.165633,4144.0,6.0,50.000000,0.0,11.0,0.545455
9,387605,Mathematik,13,51.058490,4216.0,8.0,56.250000,2.0,9.0,0.937500


In [60]:
mastery_df_german.head(10)

,user_id,title,weeks_since_first_transaction,mastery,diligence,num_questions,percentage_correct,num_review,num_view,ratio_window_visible
0,387604,Deutsch,10,0.000000,0.0,0.0,NaN,0.0,NaN,NaN
1,387604,Deutsch,11,0.000000,0.0,0.0,NaN,0.0,NaN,NaN
2,387604,Deutsch,12,0.000000,0.0,0.0,NaN,0.0,1.0,0.909091
3,387604,Deutsch,13,0.000000,0.0,0.0,NaN,0.0,NaN,0.900000
4,387604,Deutsch,14,0.000000,0.0,1.0,0.0,0.0,15.0,0.645161
5,387604,Deutsch,15,0.000000,0.0,0.0,NaN,0.0,3.0,0.214286
6,387604,Deutsch,16,0.000000,0.0,0.0,NaN,0.0,NaN,0.086957
7,387604,Deutsch,17,0.000000,8.0,4.0,12.5,0.0,7.0,0.555556
8,387604,Deutsch,19,0.173611,68.0,1.0,50.0,0.0,1.0,0.157895
9,387604,Deutsch,23,0.173611,258.0,8.0,50.0,6.0,19.0,0.583333


Notice that the current week definition does not bode well with the data, the first week for some users are not week 0(user 387604) because we have split the data into math and german, then there are missing weeks(51 weeks missing for user 387615), because this user has been absent from the platform for a really long time.

See the below cell output for examples

In [61]:
# Group by user_id get all the weeks_since_first_transaction values for each user and sort them in ascending order
mastery_df_german.groupby('user_id').weeks_since_first_transaction.unique().reset_index()

,user_id,weeks_since_first_transaction
0,387604,"[10, 11, 12, 13, 14, 15, 16, 17, 19, 23, 24, 2..."
1,387605,"[4, 6, 13, 14, 22, 26]"
2,387615,"[0, 1, 2, 4, 55]"
3,387643,"[39, 40, 41, 42, 46, 47, 52]"
4,387644,"[0, 1, 2, 3, 5, 9, 10, 11, 12, 14, 15, 16, 18,..."
...,...,...
17395,431987,[0]
17396,431989,[0]
17397,431991,[0]
17398,431999,[0]


To fix this, we change our week definition to be the "active" weeks (i.e. weeks that the user has done at least one kind of event) of a user. Meaning that in the case that a user solves some questions the first week then does not login to the platform for 5-6 weeks, and comes back, the 2nd week of that user will be considered the time where the user has returned. 

In [67]:
# We only want to consider the active weeks of the users
temp = mastery_df_german.groupby('user_id').weeks_since_first_transaction.apply(list)
mastery_df_german = mastery_df_german.join(temp, on='user_id', how='left', rsuffix='_list')
# update the weeks_since_first_transaction column to the index of the list
mastery_df_german['weeks_since_first_transaction'] = mastery_df_german.apply(lambda x: x['weeks_since_first_transaction_list'].index(x['weeks_since_first_transaction']), axis=1)
# drop the list column
mastery_df_german = mastery_df_german.drop(columns=['weeks_since_first_transaction_list'])

# We only want to consider the active weeks of the users
temp = mastery_df_math.groupby('user_id').weeks_since_first_transaction.apply(list)
mastery_df_math = mastery_df_math.join(temp, on='user_id', how='left', rsuffix='_list')
# update the weeks_since_first_transaction column to the index of the list
mastery_df_math['weeks_since_first_transaction'] = mastery_df_math.apply(lambda x: x['weeks_since_first_transaction_list'].index(x['weeks_since_first_transaction']), axis=1)
# drop the list column
mastery_df_math = mastery_df_math.drop(columns=['weeks_since_first_transaction_list'])

In [ ]:
# Find which user id belongs to the maximum weeks_since_first_transaction just for visualization
max_week = mastery_df_german.weeks_since_first_transaction.max()
user_id = mastery_df_german[mastery_df_german['weeks_since_first_transaction']==max_week].user_id
sns.lineplot(x='weeks_since_first_transaction', y='mastery', data=mastery_df_german[mastery_df_german['user_id']==user_id.values[0]])
plt.title('Mastery level change for the user with maximum weeks since first transaction - German')
plt.xlabel('Weeks Since First Transaction')
plt.ylabel('Mastery')
plt.show()

In [ ]:
# Find which user id belongs to the maximum weeks_since_first_transaction just for visualization
max_week = mastery_df_math.weeks_since_first_transaction.max()
user_id = mastery_df_math[mastery_df_math['weeks_since_first_transaction']==max_week].user_id
sns.lineplot(x='weeks_since_first_transaction', y='mastery', data=mastery_df_math[mastery_df_math['user_id']==user_id.values[0]])
plt.title('Mastery level change for the user with maximum weeks since first transaction - Math')
plt.xlabel('Weeks Since First Transaction')
plt.ylabel('Mastery')
plt.show()

Currently we have the dataframe that has users mastery level per week along with other features. But even if we are currently considering the active weeks of each user, there are still some users who have interacted with the platform so little that they don't serve much use for our case. These are the users who only have been in the platform for a few weeks and thus do not have sufficient data to be trained on. We drop these users in the below cell:

We chose the minimum required interaction count to be 4 as this still leaves us with an acceptible amount of data.

In [ ]:
# This number below changes which type of users we will consider. We will consider users who have at least this amount of weeks of activity
week_larger_than = 6

# Find the max weeks since first transaction for each user
max_weeks_math = mastery_df_math.groupby('user_id').weeks_since_first_transaction.max().reset_index()
users_with_no_week_math = max_weeks_math[max_weeks_math.weeks_since_first_transaction < week_larger_than]
# drop these users from mastery_df_2
mastery_df_subset_math = mastery_df_math[~mastery_df_math.user_id.isin(users_with_no_week_math.user_id)]
print("Dropping: ", users_with_no_week_math.user_id.nunique(), " users from math.")
print("Unique user count left in math: ", mastery_df_subset_math.user_id.nunique())

# Find the max weeks since first transaction for each user
max_weeks_german = mastery_df_german.groupby('user_id').weeks_since_first_transaction.max().reset_index()
users_with_no_week_german = max_weeks_german[max_weeks_german.weeks_since_first_transaction < week_larger_than]
# drop these users from mastery_df_2
mastery_df_subset_german = mastery_df_german[~mastery_df_german.user_id.isin(users_with_no_week_german.user_id)]
print("Dropping: ", users_with_no_week_german.user_id.nunique(), " users from german.")
print("Unique user count left in german: ", mastery_df_subset_german.user_id.nunique())

Now there is an interesting edge case left for our use case. There are some users even after interacting with the platform for multiple weeks, have obtained 0 mastery in the end. 

Because we are aiming to predict the mastery level, users who have obtained no mastery at all will not be helping us in this journey, and thus we drop them below:

In [ ]:
# Find the max mastery level per user
max_mastery_math = mastery_df_subset_math.groupby('user_id').mastery.max().reset_index()
# Drop users with 0 mastery
zero_mastery_math = max_mastery_math[max_mastery_math.mastery == 0]
print("Dropping ", zero_mastery_math.shape[0], " users with 0 mastery for math")
mastery_df_subset_math = mastery_df_subset_math[~mastery_df_subset_math.user_id.isin(zero_mastery_math.user_id)]

# Find the max mastery level per user
max_mastery_german = mastery_df_subset_german.groupby('user_id').mastery.max().reset_index()
# Drop users with 0 mastery
zero_mastery_german = max_mastery_german[max_mastery_german.mastery == 0]
print("Dropping ", zero_mastery_german.shape[0], " users with 0 mastery for german")
mastery_df_subset_german = mastery_df_subset_german[~mastery_df_subset_german.user_id.isin(zero_mastery_german.user_id)]

In [ ]:
mastery_df_math[mastery_df_math.user_id == 387604]

In [ ]:
mastery_df_german[mastery_df_german.user_id == 387604]

In [ ]:
# Plot the average mastery for each week for the first weeks we decided above
sns.lineplot(x='weeks_since_first_transaction', y='mastery', data=mastery_df_subset_math[mastery_df_subset_math.weeks_since_first_transaction < week_larger_than])
plt.title('Average Mastery Level - Math')
plt.xlabel('Weeks Since First Transaction')
plt.ylabel('Mastery')
plt.show()

In [ ]:
# Plot the average mastery for each week for the first weeks we decided above
sns.lineplot(x='weeks_since_first_transaction', y='mastery', data=mastery_df_subset_german[mastery_df_subset_german.weeks_since_first_transaction < week_larger_than])
plt.title('Average Mastery Level - German')
plt.xlabel('Weeks Since First Transaction')
plt.ylabel('Mastery')
plt.show()

---

## Task 2: Model Building

Train a model for your research question. 

### Regression

#### Math

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error

data = mastery_df_subset_math.fillna(0)

WEEK_COUNT = 6

# Filter the data to include rows with weeks_since_first_transaction <= Week count - 1
filtered_data = data[data['weeks_since_first_transaction'] <= WEEK_COUNT - 1]

# Filter the data again to get target rows with weeks_since_first_transaction = Week count
target_data = data[data['weeks_since_first_transaction'] == WEEK_COUNT]

# Define features and target variable for the regression
X = filtered_data.drop(columns=['user_id', 'title', 'mastery'])
y = filtered_data['mastery']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the models and their hyperparameters
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso()
}

alphas = np.logspace(-3, 3, 20)  # Regularization strengths for Ridge and Lasso

# Perform grid search for each model
for model_name, model in models.items():
    if model_name == 'LinearRegression':
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_test, model.predict(X_test))
        print(f"{model_name} - Mean Squared Error: {mse}")
    else:
        param_grid = {'alpha': alphas}
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(mean_squared_error))
        grid_search.fit(X_train, y_train)
        best_alpha = grid_search.best_params_['alpha']
        best_model = grid_search.best_estimator_
        mse = mean_squared_error(y_test, best_model.predict(X_test))
        print(f"{model_name} (alpha={best_alpha}) - Mean Squared Error: {mse}")



Initially, the data is preprocessed by filtering it according to the weeks_since_first_transaction column. The filtered_data dataframe comprises rows with weeks up to and including WEEK_COUNT - 1, while the target_data dataframe has rows with weeks equal to WEEK_COUNT.

Subsequently, a grid search is conducted for Ridge and Lasso to identify the best alpha value for each model. For Linear Regression, as there are no hyperparameters to optimize, the model is directly fitted to the training set.

As per the MSE values, Ridge Regression with an alpha of 1000 slightly outperforms Linear Regression, whereas Lasso Regression with an alpha of 233.572 lags behind. This indicates that Ridge Regression strikes a superior balance between model complexity and regularization, thereby offering better protection against overfitting in comparison to the other two models within this math dataset.

#### Test Step

In [ ]:
target_X = target_data.drop(columns=['user_id', 'title', 'mastery'])
target_y = target_data['mastery']

# Perform grid search for each model and store the best models
best_models = {}
for model_name, model in models.items():
    if model_name == 'LinearRegression':
        model.fit(X_train, y_train)
        best_models[model_name] = model
    else:
        param_grid = {'alpha': alphas}
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(mean_squared_error))
        grid_search.fit(X_train, y_train)
        best_alpha = grid_search.best_params_['alpha']
        best_models[model_name] = grid_search.best_estimator_

# Evaluate and print the MSE for each model on the target_data
test_mse_math = []
test_rmse_math = []
test_mae_math = []

for model_name, model in best_models.items():
    y_pred = model.predict(target_X)
    mse = mean_squared_error(target_y, y_pred)
    rmse = mean_squared_error(target_y, y_pred, squared=False)
    mae = mean_absolute_error(target_y, y_pred)

    test_mse_math.append(mse)
    test_rmse_math.append(rmse)
    test_mae_math.append(mae)

    if model_name == 'LinearRegression':
        print(f"{model_name} - Mean Squared Error: {mse}")
        print(f"{model_name} - Root Mean Squared Error: {rmse}")
        print(f"{model_name} - Mean Absolute Error: {mae}")
    else:
        alpha = model.alpha
        print(f"{model_name} (alpha={alpha}) - Mean Squared Error: {mse}")
        print(f"{model_name} (alpha={alpha}) - Root Mean Squared Error: {rmse}")
        print(f"{model_name} (alpha={alpha}) - Mean Absolute Error: {mae}")


#### Explainability

In [ ]:
# Feature names
feature_names = target_X.columns.tolist()

# Display coefficients of each model
for model_name, model in best_models.items():
    if hasattr(model, "coef_"):
        print(f"{model_name} coefficients:")
        for feature, coef in zip(feature_names, model.coef_):
            print(f"{feature}: {coef}")
    else:
        print(f"{model_name} does not have coefficients")
    print()

Now we are evaluating the performance of the three best models (Linear Regression, Ridge Regression, and Lasso Regression) on the target_data, which contains rows with weeks_since_first_transaction equal to WEEK_COUNT.

Based on these evaluation metrics, Ridge Regression with an alpha of 1000 again outperforms the other two models on the target_data. It has a lower MSE, RMSE, and MAE compared to Linear Regression and Lasso Regression. This suggests that Ridge Regression continues to provide a better balance between model complexity and regularization

#### Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

for idx, (model_name, model) in enumerate(best_models.items()):
    y_pred = model.predict(target_X)

    axs[idx].scatter(target_y, target_y, alpha=0.5, marker='o', color='blue', label='Real')
    axs[idx].scatter(target_y, y_pred, alpha=0.5, marker='x', color='red', label='Predicted')
    axs[idx].set_title(model_name)
    axs[idx].set_xlabel('Real Mastery')
    axs[idx].legend(loc='upper left')

axs[0].set_ylabel('Predicted Mastery')
plt.suptitle('Predicted Mastery vs Real Mastery - Math')
plt.tight_layout()
plt.show()


#### German

In [ ]:
data = mastery_df_subset_german.fillna(0)

# Filter the data to include rows with weeks_since_first_transaction <= Week count - 1
filtered_data = data[data['weeks_since_first_transaction'] <=  WEEK_COUNT - 1]

# Filter the data again to get target rows with weeks_since_first_transaction = Week count
target_data = data[data['weeks_since_first_transaction'] ==  WEEK_COUNT]

# Define features and target variable for the regression
X = filtered_data.drop(columns=['user_id', 'title', 'mastery'])
y = filtered_data['mastery']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the models and their hyperparameters
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso()
}

alphas = np.logspace(-3, 3, 20)  # Regularization strengths for Ridge and Lasso

# Perform grid search for each model
for model_name, model in models.items():
    if model_name == 'LinearRegression':
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_test, model.predict(X_test))
        print(f"{model_name} - Mean Squared Error: {mse}")
    else:
        param_grid = {'alpha': alphas}
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(mean_squared_error))
        grid_search.fit(X_train, y_train)
        best_alpha = grid_search.best_params_['alpha']
        best_model = grid_search.best_estimator_
        mse = mean_squared_error(y_test, best_model.predict(X_test))
        print(f"{model_name} (alpha={best_alpha}) - Mean Squared Error: {mse}")

We have applied the same approach the German dataset. As per the MSE values, Ridge Regression with an alpha of 1000 slightly outperforms Linear Regression, whereas Lasso Regression with an alpha of 233.572 lags behind. This indicates that Ridge Regression strikes a superior balance between model complexity and regularization, thereby offering better protection against overfitting in comparison to the other two models within this german dataset.

#### Test Step

In [ ]:
target_X = target_data.drop(columns=['user_id', 'title', 'mastery'])
target_y = target_data['mastery']

# Perform grid search for each model and store the best models
best_models = {}
for model_name, model in models.items():
    if model_name == 'LinearRegression':
        model.fit(X_train, y_train)
        best_models[model_name] = model
    else:
        param_grid = {'alpha': alphas}
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(mean_squared_error))
        grid_search.fit(X_train, y_train)
        best_alpha = grid_search.best_params_['alpha']
        best_models[model_name] = grid_search.best_estimator_

# Evaluate and print the MSE for each model on the target_data
test_mse_german = []
test_rmse_german = []
test_mae_german = []

for model_name, model in best_models.items():
    y_pred = model.predict(target_X)
    mse = mean_squared_error(target_y, y_pred)
    rmse = mean_squared_error(target_y, y_pred, squared=False)
    mae = mean_absolute_error(target_y, y_pred)

    test_mse_german.append(mse)
    test_rmse_german.append(rmse)
    test_mae_german.append(mae)
    if model_name == 'LinearRegression':
        print(f"{model_name} - Mean Squared Error: {mse}")
        print(f"{model_name} - Root Mean Squared Error: {rmse}")
        print(f"{model_name} - Mean Absolute Error: {mae}")
    else:
        alpha = model.alpha
        print(f"{model_name} (alpha={alpha}) - Mean Squared Error: {mse}")
        print(f"{model_name} (alpha={alpha}) - Root Mean Squared Error: {rmse}")
        print(f"{model_name} (alpha={alpha}) - Mean Absolute Error: {mae}")


#### Explainability

In [ ]:
# Feature names
feature_names = target_X.columns.tolist()

# Display coefficients of each model
for model_name, model in best_models.items():
    if hasattr(model, "coef_"):
        print(f"{model_name} coefficients:")
        for feature, coef in zip(feature_names, model.coef_):
            print(f"{feature}: {coef}")
    else:
        print(f"{model_name} does not have coefficients")
    print()

At this point, we assess the performance of the top three models (Linear Regression, Ridge Regression, and Lasso Regression) using the target_data, which includes rows where weeks_since_first_transaction is equal to WEEK_COUNT.

From the evaluation metrics, Linear Regression surpasses the other two models when applied to the target_data. It exhibits lower values for MSE, RMSE, and MAE in comparison to Ridge Regression and Lasso Regression. 

#### Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

for idx, (model_name, model) in enumerate(best_models.items()):
    y_pred = model.predict(target_X)

    axs[idx].scatter(target_y, target_y, alpha=0.5, marker='o', color='blue', label='Real')
    axs[idx].scatter(target_y, y_pred, alpha=0.5, marker='x', color='red', label='Predicted')
    axs[idx].set_title(model_name)
    axs[idx].set_xlabel('Real Mastery')
    axs[idx].legend(loc='upper left')

axs[0].set_ylabel('Predicted Mastery')
plt.suptitle('Predicted Mastery vs Real Mastery - German')
plt.tight_layout()
plt.show()


---

### LSTM and GRU models

For LSTM and GRU models we created a **lstm_functions.py** script to define two deep learning models, an LSTM (Long Short-Term Memory) model and a GRU (Gated Recurrent Unit) model, along with a custom dataset class for loading and pre-processing the data. The models are intended to predict the mastery variable for a target week, given the input sequence of data from the preceding weeks.

#### Load Necessary Libraries

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2" # limiting to one GPU
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### German

#### LSTM

##### Step 1: Creation of test and train dataframes

In [ ]:
# Fill NaN values with 0
mastery_df_subset_german.fillna(0, inplace=True)

# Normalize the data
scaler = MinMaxScaler()
mastery_df_subset_german[['diligence','num_questions', 'percentage_correct', 'num_review', 'num_view', 'ratio_window_visible']] = scaler.fit_transform(mastery_df_subset_german[['diligence', 'num_questions', 'percentage_correct', 'num_review', 'num_view', 'ratio_window_visible']])

In [ ]:
user_ids = mastery_df_subset_german['user_id'].unique()
valid_user_ids = []

for user_id in user_ids:
    user_data = mastery_df_subset_german[mastery_df_subset_german['user_id'] == user_id]
    if len(user_data) > 6:
        valid_user_ids.append(user_id)

valid_mastery_df_german = mastery_df_subset_german[mastery_df_subset_german['user_id'].isin(valid_user_ids)]

train_user_ids, test_user_ids = train_test_split(valid_user_ids, test_size=0.2, random_state=42)
train_df = valid_mastery_df_german[valid_mastery_df_german['user_id'].isin(train_user_ids)]
test_df = valid_mastery_df_german[valid_mastery_df_german['user_id'].isin(test_user_ids)]

print("Train data frame unique user count:", train_df['user_id'].nunique())
print("Test data frame unique user count:", test_df['user_id'].nunique())

In [ ]:
from lstm_functions import MasteryDataset

train_dataset = MasteryDataset(train_df)
test_dataset = MasteryDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

print(len(train_dataset))
print(len(test_dataset))

**MasteryDataset** class: This class is a custom PyTorch dataset that takes a pandas dataframe df and a target_week as input. The dataset is designed to provide the input sequences and corresponding target values for the LSTM and GRU models.
- The **_get_filtered_user_ids** method filters out user_ids that have data for the target week.
- The __getitem__ method returns an input sequence and the target value for a given index. It pads the input sequence with zeros if necessary to match the target week's length and creates tensors from the input sequence and target value.

In [ ]:
from lstm_functions import LSTMModel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = train_dataset[0][0].shape[1]
hidden_size = 64
num_layers = 4
output_size = 1
learning_rate = 0.0001

model = LSTMModel(input_size, hidden_size, num_layers, output_size, device).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_params = np.sum(np.fromiter((p.numel() for p in model.parameters() if p.requires_grad), dtype=int))
print(f'The model has {num_params:,} trainable parameters')

**LSTMModel** class: This class defines an LSTM model that inherits from PyTorch's nn.Module class. It takes the input size, hidden size, number of layers, output size, and a device as input arguments.
- The LSTM layers are defined using the nn.LSTM module with the given input size, hidden size, and number of layers.
- Two fully connected linear layers (nn.Linear) are added after the LSTM layers.
- The **forward** method takes an input tensor x, initializes the hidden and cell states h0 and c0, and passes the input through the LSTM layers, followed by the linear layers with a ReLU activation function in between.

#### Step 2: Train the model

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

#### Step 3: Test the model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
predictions_german_lstm = []
test_loss = 0
num_test_samples = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions_german_lstm.extend(outputs.cpu().numpy().tolist())

real_values_german = test_df[test_df["weeks_since_first_transaction"] == 6]
# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_german["mastery"], predictions_german_lstm)
rmse = mean_squared_error(real_values_german["mastery"], predictions_german_lstm, squared =False)
mae = mean_absolute_error(real_values_german["mastery"], predictions_german_lstm)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)



After obtaining the predictions, the ground truth values for the test dataset are extracted from the test_df dataframe for rows where the weeks_since_first_transaction value is 6.

#### GRU

##### Step 1: Build the model

In [ ]:
from lstm_functions import GRUModel
model = GRUModel(input_size, hidden_size, num_layers, output_size, device).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_params = np.sum(np.fromiter((p.numel() for p in model.parameters() if p.requires_grad), dtype=int))
print(f'The model has {num_params:,} trainable parameters')

**GRUModel** class: This class defines a GRU model, similar to the **LSTMModel** class, but it uses the nn.GRU module instead of the nn.LSTM module. The GRU model also inherits from PyTorch's nn.Module class and takes the same input arguments as the LSTM model.
- The GRU layers are defined using the nn.GRU module with the given input size, hidden size, and number of layers.
- Two fully connected linear layers (nn.Linear) are added after the GRU layers.
- The forward method takes an input tensor x, initializes the hidden state h0, and passes the input through the GRU layers, followed by the linear layers with a ReLU activation function in between.

##### Step 2: Train the model

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

##### Step 3: Test the model

In [ ]:
model.eval()
predictions_german_gru = []
test_loss = 0
num_test_samples = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions_german_gru.extend(outputs.cpu().numpy().tolist())

# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_german["mastery"], predictions_german_gru)
rmse = mean_squared_error(real_values_german["mastery"], predictions_german_gru, squared =False)
mae = mean_absolute_error(real_values_german["mastery"], predictions_german_gru)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)

##### If all predictions are 0

In [ ]:
predictions = [0] * len(test_dataset)

# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_german["mastery"], predictions)
rmse = mean_squared_error(real_values_german["mastery"], predictions, squared =False)
mae = mean_absolute_error(real_values_german["mastery"], predictions)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)

The results presented above are based on a model that predicts all zeros for the test dataset, essentially serving as a naive baseline. The performance of this baseline model is evaluated using three metrics: 
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE). 

With an MSE of 598.292, an RMSE of 24.460, and an MAE of 20.287, the baseline model demonstrates a significant error when predicting the 'mastery' variable.

### Math

#### LSTM

#### Step 1: Build the model

In [ ]:
# Fill NaN values with 0
mastery_df_subset_math.fillna(0, inplace=True)

# Normalize the data
scaler = MinMaxScaler()
mastery_df_subset_math[['diligence', 'num_questions', 'percentage_correct', 'num_review', 'num_view', 'ratio_window_visible']] = scaler.fit_transform(mastery_df_subset_math[['diligence', 'num_questions', 'percentage_correct', 'num_review', 'num_view', 'ratio_window_visible']])

In [ ]:
user_ids = mastery_df_subset_math['user_id'].unique()
valid_user_ids = []

for user_id in user_ids:
    user_data = mastery_df_subset_math[mastery_df_subset_math['user_id'] == user_id]
    if len(user_data) > 6:
        valid_user_ids.append(user_id)

valid_mastery_df_math = mastery_df_subset_math[mastery_df_subset_math['user_id'].isin(valid_user_ids)]

train_user_ids, test_user_ids = train_test_split(valid_user_ids, test_size=0.2, random_state=42)
train_df = valid_mastery_df_math[valid_mastery_df_math['user_id'].isin(train_user_ids)]
test_df = valid_mastery_df_math[valid_mastery_df_math['user_id'].isin(test_user_ids)]

print("Train data frame unique user count:", train_df['user_id'].nunique())
print("Test data frame unique user count:", test_df['user_id'].nunique())

In [ ]:
from lstm_functions import MasteryDataset

train_dataset = MasteryDataset(train_df)
test_dataset = MasteryDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

print(len(train_dataset))
print(len(test_dataset))

In [ ]:
from lstm_functions import LSTMModel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = train_dataset[0][0].shape[1]
hidden_size = 64
num_layers = 4
output_size = 1
learning_rate = 0.0001

model = LSTMModel(input_size, hidden_size, num_layers, output_size, device).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_params = np.sum(np.fromiter((p.numel() for p in model.parameters() if p.requires_grad), dtype=int))
print(f'The model has {num_params:,} trainable parameters')

#### Step 2: Train the model

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

#### Step 3: Test the model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
predictions_math_lstm = []
test_loss = 0
num_test_samples = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions_math_lstm.extend(outputs.cpu().numpy().tolist())

real_values_math = test_df[test_df["weeks_since_first_transaction"] == 6]
# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_math["mastery"], predictions_math_lstm)
rmse = mean_squared_error(real_values_math["mastery"], predictions_math_lstm, squared =False)
mae = mean_absolute_error(real_values_math["mastery"], predictions_math_lstm)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)

#### GRU

##### Step 1: Build the model

In [ ]:
from lstm_functions import GRUModel
model = GRUModel(input_size, hidden_size, num_layers, output_size, device).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_params = np.sum(np.fromiter((p.numel() for p in model.parameters() if p.requires_grad), dtype=int))
print(f'The model has {num_params:,} trainable parameters')

##### Step 2: Train the model

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

##### Step 3: Test the model

In [ ]:
model.eval()
predictions_math_gru = []
test_loss = 0
num_test_samples = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions_math_gru.extend(outputs.cpu().numpy().tolist())

# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_math["mastery"], predictions_math_gru)
rmse = mean_squared_error(real_values_math["mastery"], predictions_math_gru, squared =False)
mae = mean_absolute_error(real_values_math["mastery"], predictions_math_gru)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)

##### If all predictions are 0

In [ ]:
predictions = [0] * len(test_dataset)

# Calculate the Mean Squared Error
mse = mean_squared_error(real_values_math["mastery"], predictions)
rmse = mean_squared_error(real_values_math["mastery"], predictions, squared =False)
mae = mean_absolute_error(real_values_math["mastery"], predictions)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)

The results presented above are based on a model that predicts all zeros for the test dataset, essentially serving as a naive baseline. The performance of this baseline model is evaluated using three metrics: 
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE). 

With an MSE of 392.008, an RMSE of 19.799, and an MAE of 15.227, the baseline model demonstrates a significant error when predicting the 'mastery' variable.

*Your discussion about your model training goes here*

**DISCUSSION**: In the regression section, we initially focused on data preprocessing and filtering. The data was filtered based on the 'weeks_since_first_transaction' column to create two separate dataframes: one with rows where the weeks are less than or equal to WEEK_COUNT - 1 (filtered_data) and another with rows where the weeks equal WEEK_COUNT (target_data). Next, we defined the features and target variable for the regression task, excluding 'user_id', 'title', and 'mastery' columns from the feature set. The data was then split into training and testing sets using a 70/30 split.

In the LSTM and GRU section, we first created a custom PyTorch dataset class, MasteryDataset, to handle the data preparation and processing for our time series data. This class is responsible for filtering valid user_ids, handling variable-length sequences by padding with zeros, and returning the input sequence (first six weeks) and target value (mastery at week 6) for each user.

Next, we defined two PyTorch model classes: LSTMModel and GRUModel. Both classes contain an RNN layer (LSTM or GRU, respectively), followed by two linear layers with a ReLU activation function after the first linear layer. These models are designed to accept input sequences and output a single value representing the predicted mastery at week 6.

After defining the models, we trained the LSTM model using the Mean Squared Error (MSE) loss as the objective function and the Adam optimizer. The model was evaluated on the test set using a DataLoader, which allowed us to process test data in batches. The evaluation metrics used for the LSTM model include Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and Mean Absolute Error (MAE).

A similar process was carried out for the GRU model, using the same training parameters, evaluation metrics, and test set. 



---

## Task 3: Model Evaluation
In this task, you will use metrics to evaluate your model.

**METRICS**

Mean Squared Error (MSE), Mean Absolute Error (MAE), and Root Mean Squared Error (RMSE) are popular evaluation metrics for regression tasks because they provide valuable insights into the performance of a model in predicting continuous outcomes. Each of these metrics has its unique advantages and contributes to the overall understanding of model performance.

- **Mean Squared Error (MSE)**: MSE measures the average squared difference between the predicted values and the actual values. By squaring the errors, MSE puts more emphasis on larger errors and penalizes them heavily. 
- **Mean Absolute Error (MAE)**: MAE calculates the average absolute difference between the predicted values and the actual values. This metric provides a more interpretable measure of the average error magnitude, as it is on the same scale as the target variable.
- **Root Mean Squared Error (RMSE)**: This metric combines the sensitivity to large errors (as in MSE) with the interpretability of the error scale (as in MAE). RMSE represents the standard deviation of the residuals (prediction errors) and provides an easily understandable measure of the average error magnitude.

### Regression Models

#### Math

In [ ]:
#Plot and compare the test performances of regressions
reg_labels = ['Linear', 'Ridge', 'Lasso']

#Mean Squared Error
fig, axs = plt.subplots(1, 3, figsize=(10, 4))
axs[0].bar(reg_labels, test_mse_math, color=['darkblue', 'darkred', 'darkorange'])
axs[0].set_title('MSE By Regression Type')
axs[0].set_xlabel('Type of Regression')
axs[0].set_ylabel('Mean Squared Error')

#Root Mean Squared Error
axs[1].bar(reg_labels, test_rmse_math, color=['darkblue', 'darkred', 'darkorange'])
axs[1].set_title('RMSE By Regression Type')
axs[1].set_xlabel('Type of Regression')
axs[1].set_ylabel('Root Mean Squared Error')

#Mean Absolute Error
axs[2].bar(reg_labels, test_mae_math, color=['darkblue', 'darkred', 'darkorange'])
axs[2].set_title('MAE By Regression Type')
axs[2].set_xlabel('Type of Regression')
axs[2].set_ylabel('Mean Absolute Error')

plt.suptitle("Evaluation Metrics - Math", fontsize = 14)
plt.tight_layout()

#### German

In [ ]:
#Plot and compare the test performances of regressions
reg_labels = ['Linear', 'Ridge', 'Lasso']

#Mean Squared Error
fig, axs = plt.subplots(1, 3, figsize=(10, 4))
axs[0].bar(reg_labels, test_mse_german, color=['darkblue', 'darkred', 'darkorange'])
axs[0].set_title('MSE')
axs[0].set_xlabel('Type of Regression')
axs[0].set_ylabel('Mean Squared Error')

#Root Mean Squared Error
axs[1].bar(reg_labels, test_rmse_german, color=['darkblue', 'darkred', 'darkorange'])
axs[1].set_title('RMSE')
axs[1].set_xlabel('Type of Regression')
axs[1].set_ylabel('Root Mean Squared Error')

#Mean Absolute Error
axs[2].bar(reg_labels, test_mae_german, color=['darkblue', 'darkred', 'darkorange'])
axs[2].set_title('MAE')
axs[2].set_xlabel('Type of Regression')
axs[2].set_ylabel('Mean Absolute Error')

plt.suptitle("Evaluation Metrics - German", fontsize = 14)
plt.tight_layout()

Following the data preparation, we experimented with three regression models: Linear Regression, Ridge Regression, and Lasso Regression. For Ridge and Lasso Regression, we performed a grid search to identify the optimal alpha (regularization strength) values for each model. The performance of each model was evaluated based on the Mean Squared Error (MSE) on the testing set. 

**Ridge Regression** with an alpha of 1000 demonstrated a slightly better performance compared to Linear Regression, while Lasso Regression with an alpha of 233.572 yielded the worst performance for Math dataset. 

In the German dataset, **Linear Regression** demonstrated a slightly better performance compared to Ridge Regression while Lasso Regression with an alpha of 233.572 yielded the worst performance

### LSTM

#### Math

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Create a scatter plot
ax1.scatter(real_values_math["mastery"], predictions_math_lstm)
ax1.set_xlabel("Real Values")
ax1.set_ylabel("Predicted Values")
ax1.set_title("Scatter plot of Real vs Predicted Mastery Values")

# Create a line plot
ax2.plot(real_values_math["mastery"].values, label="Real Values")
ax2.plot(predictions_math_lstm, label="Predicted Values", linestyle="--")
ax2.set_xlabel("Sample Index")
ax2.set_ylabel("Mastery")
ax2.set_title("Line plot of Real vs Predicted Mastery Values")
ax2.legend()

plt.suptitle("LSTM - Math", fontsize = 16)
plt.tight_layout()
plt.show()

#### German

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Create a scatter plot
ax1.scatter(real_values_german["mastery"], predictions_german_lstm)
ax1.set_xlabel("Real Values")
ax1.set_ylabel("Predicted Values")
ax1.set_title("Scatter plot of Real vs Predicted Mastery Values")

# Create a line plot
ax2.plot(real_values_german["mastery"].values, label="Real Values")
ax2.plot(predictions_german_lstm, label="Predicted Values", linestyle="--")
ax2.set_xlabel("Sample Index")
ax2.set_ylabel("Mastery")
ax2.set_title("Line plot of Real vs Predicted Mastery Values")
ax2.legend()

plt.suptitle("LSTM - German", fontsize = 16)
plt.tight_layout()
plt.show()

### GRU

#### Math

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Create a scatter plot
ax1.scatter(real_values_math["mastery"], predictions_math_gru)
ax1.set_xlabel("Real Values")
ax1.set_ylabel("Predicted Values")
ax1.set_title("Scatter plot of Real vs Predicted Mastery Values")

# Create a line plot
ax2.plot(real_values_math["mastery"].values, label="Real Values")
ax2.plot(predictions_math_gru, label="Predicted Values", linestyle="--")
ax2.set_xlabel("Sample Index")
ax2.set_ylabel("Mastery")
ax2.set_title("Line plot of Real vs Predicted Mastery Values")
ax2.legend()

plt.suptitle("GRU - Math", fontsize = 16)
plt.tight_layout()
plt.show()

#### German

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Create a scatter plot
ax1.scatter(real_values_german["mastery"], predictions_german_gru)
ax1.set_xlabel("Real Values")
ax1.set_ylabel("Predicted Values")
ax1.set_title("Scatter plot of Real vs Predicted Mastery Values")

# Create a line plot
ax2.plot(real_values_german["mastery"].values, label="Real Values")
ax2.plot(predictions_german_gru, label="Predicted Values", linestyle="--")
ax2.set_xlabel("Sample Index")
ax2.set_ylabel("Mastery")
ax2.set_title("Line plot of Real vs Predicted Mastery Values")
ax2.legend()

plt.suptitle("GRU - German", fontsize = 16)
plt.tight_layout()
plt.show()

**DISCUSSION**

**MATH**

Baseline model (Prediction of full 0s):
- Mean Squared Error: 392.00874191510013
- Root Mean Squared Error: 19.799210638687093
- Mean Absolute Error: 15.227262151480423

For the Math dataset, the LSTM and GRU models once again outperform the baseline model by a significant margin. The LSTM model yields a Mean Squared Error (MSE) of 18.17, a Root Mean Squared Error (RMSE) of 4.26, and a Mean Absolute Error (MAE) of 2.49. Meanwhile, the GRU model has an MSE of 20.40, an RMSE of 4.52, and an MAE of 2.68.

In stark contrast, the baseline model demonstrates much higher error values, with an MSE of 392.01, an RMSE of 19.80, and an MAE of 15.23. The substantial improvement in prediction accuracy provided by the LSTM and GRU models can be attributed to their ability to effectively capture temporal patterns and dependencies in the dataset. These results further highlight the effectiveness of LSTM and GRU models for predicting mastery at week 6, in comparison to the baseline model.

**GERMAN**

Baseline Model (Prediction of full 0s):
- Mean Squared Error: 598.2922590590904
- Root Mean Squared Error: 24.460013472177206
- Mean Absolute Error: 20.286615810758597

In the German dataset, the LSTM and GRU models show significantly better performance than the baseline model. The LSTM model achieves a Mean Squared Error (MSE) of 21.73, a Root Mean Squared Error (RMSE) of 4.66, and a Mean Absolute Error (MAE) of 3.20. The GRU model has an MSE of 23.24, an RMSE of 4.82, and an MAE of 3.11.

In comparison, the baseline model exhibits much higher error values, with an MSE of 598.29, an RMSE of 24.46, and an MAE of 20.29. This considerable difference in performance indicates that the LSTM and GRU models are more effective in predicting mastery at week 6 than the baseline model. Both LSTM and GRU models demonstrate their ability to capture temporal patterns and dependencies in the data, leading to substantially improved prediction accuracy.

---

## Task 4: Team Reflection
Please describe the contributions of each team member to Milestone 4. Reflect on how you worked as team: what went well, what can be improved for the next milestone?

For this milestone, the members performed the following tasks:

**Aybars Yazici:** Data exploration and preprocessing, extracting mastery levels and the majority of the features.

**Ilker Gul:** Implementation of Regression, LSTM, and GRU models. Training, testing, and evaluating the models on the given dataset. Visualization of the results.

**Can Kirimca:** Evaluation of the models and visualization of the results. Extraction of a few features and separating the user records into Math and German.

Although different members worked on different components of the project, we frequently held meetings to discuss the intermediate results and consulted each member on each step. 

Since LSTM and GRU have large numbers of parameters, our main concern was that the amount of data might be insufficient to train those models. However, these models performed better than we expected and managed the capture the sequential patterns in our data, and as previously stated, they performed significantly better than regression models. One potential task for the future might be to perform data augmentation to train these models on larger data as we expect them to perform better on larger datasets.

Furthermore, one of the possible reasons why the regressions performed poorly is that we need to find features that describe the patterns in the data. One of our strategies for the next milestone will be to explore the dataset further and extract more meaningful features to improve the predictive performance of each model used.